## What Are Decorators?

### Basic exampel


In [9]:
import time

def timer(func): #The decorator function 
    def wrapper(*args, **kwargs):    #The wrapper 
        start = time.time()
        result = func(*args, **kwargs) # Wrapped function call
        end = time.time()
        print(f"{func.__name__} executed in {end - start:.4f} seconds.")
        return result
    return wrapper

@timer
def calculate_sum(n):
    # time.sleep(20)          #Uncomment this to see the difference
    return sum(range(n))




calculate_sum(10**6)


calculate_sum executed in 40.0339 seconds.


499999500000

### Parameterized Example


In [59]:
import time
from functools import wraps

def retry(times, delay=1):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(times):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Retry {i+1}/{times} failed: {e}")
                    time.sleep(delay)
            raise Exception("Function failed after retries")
        return wrapper
    return decorator

@retry(times=3, delay=2)
def unstable_function():
    if time.time() % 2 > 1.5:  # Randomly fail
        raise ValueError("Random failure")
    return "Success!"

print(unstable_function())

Retry 1/3 failed: Random failure
Retry 2/3 failed: Random failure
Retry 3/3 failed: Random failure


Exception: Function failed after retries

### Time for first drill!
`Problem`: Create a decorator that takes a parameter to customize the greeting message. The decorator should add the custom greeting message before the return value of a function.

`Instructions`:

- Create a function get_name() that returns the name passed to it.
- Write a decorator custom_greeting(greeting) that adds a custom greeting message before the name.
- Apply the custom_greeting() decorator to the get_name() function and test it with different greetings.


In [ ]:
#Place your code here


In [1]:
# Use your decorator here
def get_name(name):
    return name

# Example Output 1: "Good Morning, Alice"
# Example Output 2: "Hi, Bob"
print(get_name("Alice"))
print(get_name("Bob"))


Alice
Bob


##  Examples of Built-In Decorators

### @functools.wraps
- Ensures that the decorated function retains its original metadata (e.g., name, docstring, etc.).
- Useful when creating custom decorators.

In [10]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Wrapper executed!")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def greet():
    """This function greets the user."""
    print("Hello!")

print(greet.__name__) 
print(greet.__doc__)  

greet
This function greets the user.


Now try it without @wraps(func)

In [11]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Wrapper executed!")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def greet():
    """This function greets the user."""
    print("Hello!")

print(greet.__name__) 
print(greet.__doc__)

wrapper
None


### @property
- Allows methods to be accessed like attributes.


In [12]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    @property
    def area(self):
        return self.width * self.height

rect = Rectangle(4, 5)
print(rect.area)  


20


In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def area(self):
        return 3.14 * (self._radius ** 2)

circle = Circle(5)
print(circle.area)  # 78.5


 ### @functools.lru_cache
 - Caches function outputs for optimization.


In [47]:
from functools import lru_cache

@lru_cache(maxsize=100)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))


55


### @dataclasses.dataclass
- A decorator for creating classes with minimal boilerplate.
- Automatically adds methods like __init__, __repr__, and __eq__.

In [49]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

p1 = Point(1, 2)
p2 = Point(1, 2)
print(p1)        
print(p1 == p2)   

Point(x=1, y=2)
True


Exampel without decorator


In [56]:
from dataclasses import dataclass

class Point:
    def __init__(self,x: int,y: int):
        self.x = x
        self.y = y

    def __eq__(self, other): #Try to comment this method to see the difference
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False
    

    def __repr__(self): #Try to comment this method to see the difference
        return f"Point(x={self.x}, y={self.y})"
    

p1 = Point(1, 2)
p2 = Point(1, 2)

print(p1 == p2)   
print(p1)         

True


 ## Chaining Multiple Decorators

In [69]:
def uppercase(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs).upper()
    return wrapper

def exclaim(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) + "!"
    return wrapper

@uppercase #Try to comment
@exclaim
def greet(name):
    return f"Hello, {name}"

print(greet("Alice"))  


HELLO, ALICE!


## Decorators in Classes



In [ ]:
def check_positive(func):     #The decorator is declared outside the class.
    def wrapper(self, value):
        if value < 0:
            raise ValueError("Value must be positive!")
        return func(self, value)
    return wrapper

class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance

    @check_positive  #The decorator is used as a wraper around class method 
    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited {amount}, new balance: {self.balance}")

account = BankAccount()
account.deposit(100)
# account.deposit(-50)  # Raises ValueError


## Class-Based Decorator


### Explanation:
- **Initialization (__init__)**: The decorator class takes the target function as an argument and stores it in the instance (e.g., self.func).
- **Callability (__call__)**: The __call__ method makes the class instance callable, similar to a function. It allows adding logic before and after invoking the wrapped function.
- **State Initialization (call_count)**: The decorator has a *call_count* attribute, which tracks the number of times the decorated function is invoked. 
- **State Update in __call__**: Each time the decorated function is called, the *call_count* is incremented, and the count is printed.
- **Function Execution**: The actual decorated function (*self.func*) is called with the provided arguments.

In [70]:
class CallCounter:
    def __init__(self, func):
        self.func = func
        self.call_count = 0  # State to track the number of calls

    def __call__(self, *args, **kwargs):
        self.call_count += 1  # Increment the call count
        print(f"Call {self.call_count} to {self.func.__name__}")
        return self.func(*args, **kwargs)

@CallCounter
def greet(name):
    print(f"Hello, {name}!")

# Using the decorated function
greet("Alice")
greet("Bob")
greet("Charlie")


Call 1 to greet
Hello, Alice!
Call 2 to greet
Hello, Bob!
Call 3 to greet
Hello, Charlie!


## Real-World examples
### 1. Logging Decorator:



In [71]:
def log(func):
    def wrapper(*args, **kwargs):
        print(f"Function {func.__name__} called with {args}, {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@log
def process_data(data):
    print(f"Processing {data}")

process_data("data.csv")


Function process_data called with ('data.csv',), {}
Processing data.csv


### 2. Authentication Decorator:


In [72]:
def authenticate(user_role):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if user_role != "admin":
                print("Access Denied!")
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator

@authenticate("user")
def delete_record():
    print("Record deleted.")



delete_record()


Access Denied!


In [73]:
@authenticate("admin")
def delete_record():
    print("Record deleted.")



delete_record()

Record deleted.


### 3. Rate Limiting
Prevent a function from being called more than a specified number of times within a time window.

In [74]:
import time
from functools import wraps

def rate_limit(calls_per_second):
    interval = 1.0 / calls_per_second
    def decorator(func):
        last_called = [0.0]  # Mutable to retain state

        @wraps(func)
        def wrapper(*args, **kwargs):
            elapsed = time.time() - last_called[0]
            if elapsed < interval:
                time.sleep(interval - elapsed)
            last_called[0] = time.time()
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limit(2)  # Limit to 2 calls per second
def say_hello():
    print("Hello!")

for _ in range(8):
    say_hello()


Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Hello!


# It's time to try stuff yourself!!!

We have a database whose structure is described using classes inherited from `Base`.


![db_schema_diagram.png](db_schema_diagram.png)

In [2]:
from sqlalchemy import create_engine, Column, Integer,Float, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from functools import wraps


In [3]:

Base = declarative_base()

user_group = Table(
    'user_group',
    Base.metadata,
    Column('user_id', Integer, ForeignKey('users.id'), primary_key=True),
    Column('group_id', Integer, ForeignKey('groups.group_id'), primary_key=True),
)

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True,nullable=False)
    firstname = Column(String,nullable=False)
    lastname = Column(String,nullable=False)
    company = Column(String)
    business_unit = Column(String)
    team = Column(String)
    role = Column(String)
    email = Column(String,nullable=False)
    linkedin = Column(String)
    email_count = Column(Integer, default=0)
    email_opened= Column(Integer, default=0)
    link_clicked = Column(Integer, default=0)
    submited_data = Column(Integer, default=0)
    email_reported = Column(Integer, default=0)
    templates = relationship('Template', back_populates='user')
    groups = relationship('Group', secondary=user_group, back_populates='users')

class Group(Base):
    __tablename__ = 'groups'

    group_id = Column(Integer, primary_key=True)
    name = Column(String,nullable=False)

    users = relationship('User', secondary=user_group, back_populates='groups')

class Template(Base):
    __tablename__ = 'templates'

    template_id = Column(Integer, primary_key=True)
    html_template = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship('User', back_populates='templates')

class Theme(Base):
    __tablename__ = 'themes'

    id = Column(Integer, primary_key=True)
    name = Column(String,unique=True)
    landing_page = Column(String)
    system_instruction = Column(String)
    structur_instruction = Column(String)
    prompt_template = Column(String)
    subject = Column(String)
    config_id = Column(Integer, ForeignKey('generative_config.id'))

    generative_config = relationship('GenerativeConfig', back_populates='themes')

class GenerativeConfig(Base):
    __tablename__ = 'generative_config'

    id = Column(Integer, primary_key=True)
    temperature = Column(Float)
    p = Column(Float)
    k = Column(Integer)
    max_output_tokens = Column(Integer)
    response_mime_type = Column(String)

    themes = relationship('Theme', back_populates='generative_config')

class Campaign(Base):
    __tablename__ = 'campaigns'

    id = Column(Integer, primary_key=True)
    name = Column(String,nullable=False,unique=True)
    group_id = Column(Integer, ForeignKey('groups.group_id'))
    theme_id = Column(Integer, ForeignKey('themes.id'))
    group = relationship('Group')
    theme = relationship('Theme')

    





/var/folders/hz/b_vn9fhn1r976d9tyb3flfvh0000gp/T/ipykernel_9050/3012386525.py:1: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


We also have a class `DBWorker` that implements interaction with this database using SQLAlchemy. This class was written by a programmer who knows nothing about the DRY (Don't Repeat Yourself) principle or decorators.


In [5]:

class DBWorker:
    def __init__(self, db_path='WSDrills.db'):
        self.db_path = db_path
        self.engine = create_engine(f'sqlite:///{self.db_path}')
        self.Session = sessionmaker(bind=self.engine)
        Base.metadata.create_all(self.engine)

    def add_user(self, firstname, lastname, company=None, business_unit=None, team=None, role=None, email=None, linkedin=None):
        """Add a new user to the database"""
        session = self.Session()
        try:
            new_user = User(firstname=firstname, lastname=lastname, company=company,
                            business_unit=business_unit, team=team, role=role, email=email, linkedin=linkedin)
            session.add(new_user)
            session.commit()
            session.refresh(new_user)
            return new_user
        except Exception as e:
            session.rollback()
            print(f"Error during adding user: {e}")
        finally:
            session.close()

    def delete_user(self, user_id):
        """Delete a user from the database"""
        session = self.Session()
        try:
            user = session.query(User).filter_by(id=user_id).first()
            if user:
                session.delete(user)
                session.commit()
            return user
        except Exception as e:
            session.rollback()
            print(f"Error during deleting user: {e}")
        finally:
            session.close()

    def get_user(self, user_id):
        """Get a user from the database"""
        session = self.Session()
        try:
            user = session.query(User).filter_by(id=user_id).first()
            return user
        except Exception as e:
            print(f"Error during fetching user: {e}")
        finally:
            session.close()


    def add_group(self, name):
        """Add a new group to the database"""
        session = self.Session()
        try:
            new_group = Group(name=name)
            session.add(new_group)
            session.commit()
            session.refresh(new_group)
            return new_group
        except Exception as e:
            session.rollback()
            print(f"Error during adding group: {e}")
        finally:
            session.close()

    def delete_group(self, group_id):
        """Delete a group from the database"""
        session = self.Session()
        try:
            group = session.query(Group).filter_by(group_id=group_id).first()
            if group:
                session.delete(group)
                session.commit()
            return group
        except Exception as e:
            session.rollback()
            print(f"Error during deleting group: {e}")
        finally:
            session.close()

    def get_group(self, group_id):
        """Get a group from the database"""
        session = self.Session()
        try:
            group = session.query(Group).filter_by(group_id=group_id).first()
            return group
        except Exception as e:
            print(f"Error during fetching group: {e}")
        finally:
            session.close()


    def add_template(self, html_template, user_id):
        """Add a new template to the database"""
        session = self.Session()
        try:
            new_template = Template(html_template=html_template, user_id=user_id)
            session.add(new_template)
            session.commit()
            session.refresh(new_template)
            return new_template
        except Exception as e:
            session.rollback()
            print(f"Error during adding template: {e}")
        finally:
            session.close()

    def delete_template(self, template_id):
        """Delete a template from the database"""
        session = self.Session()
        try:
            template = session.query(Template).filter_by(template_id=template_id).first()
            if template:
                session.delete(template)
                session.commit()
            return template
        except Exception as e:
            session.rollback()
            print(f"Error during deleting template: {e}")
        finally:
            session.close()

    def get_template(self, template_id):
        """Get a template from the database"""
        session = self.Session()
        try:
            template = session.query(Template).filter_by(template_id=template_id).first()
            return template
        except Exception as e:
            print(f"Error during fetching template: {e}")
        finally:
            session.close()



    def add_campaign(self, name, group_id, theme_id):
        """Add a new campaign to the database"""
        session = self.Session()
        try:
            new_campaign = Campaign(name=name, group_id=group_id, theme_id=theme_id)
            session.add(new_campaign)
            session.commit()
            session.refresh(new_campaign)
            return new_campaign
        except Exception as e:
            session.rollback()
            print(f"Error during adding campaign: {e}")
        finally:
            session.close()

    def delete_campaign(self, campaign_id):
        """Delete a campaign from the database"""
        session = self.Session()
        try:
            campaign = session.query(Campaign).filter_by(id=campaign_id).first()
            if campaign:
                session.delete(campaign)
                session.commit()
            return campaign
        except Exception as e:
            session.rollback()
            print(f"Error during deleting campaign: {e}")
        finally:
            session.close()

    def get_campaign(self, campaign_id):
        """Get a campaign from the database"""
        session = self.Session()
        try:
            campaign = session.query(Campaign).filter_by(id=campaign_id).first()
            return campaign
        except Exception as e:
            print(f"Error during fetching campaign: {e}")
        finally:
            session.close()



In [6]:

# Usage
db_worker = DBWorker()

# Adding a user
new_user = db_worker.add_user("John", "Doe", email="john.doe@example.com")

# Adding a group
new_group = db_worker.add_group("Marketing")

# Adding a template
new_template = db_worker.add_template("<h1>Welcome!</h1>", user_id=new_user.id)

# Adding a campaign
new_campaign = db_worker.add_campaign("Email Campaign 1", group_id=new_group.group_id, theme_id=1)

### Our task is to rewrite the class using decorators and following  the DRY (Don't Repeat Yourself) principle. 


- HINT: Ideally, the class should contain three methods: `add_record`, `delete_record`, and `get_record`.

In [ ]:
# Place your code here 
class DBWorker:
    def __init__(self, db_path='WSDrills.db'):
        self.db_path = db_path
        self.engine = create_engine(f'sqlite:///{self.db_path}')
        self.Session = sessionmaker(bind=self.engine)
        Base.metadata.create_all(self.engine)

    # Place your code here 

    
